In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc

os.getcwd()

# To check about the 2019 weekly file path df week filter for the rolling later

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Migration_JL_status_change'

In [2]:
prior_quarter_End=datetime.date(2018,8,4)
recent_complete_quarter_End=prior_quarter_End+datetime.timedelta(days=7*13)

print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2018-08-04
recent_complete_quarter_End 2018-11-03
lapsed_date_begin 2014-07-29


In [3]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]
lapsed=lapsed[lapsed['transaction_date']<=str(prior_quarter_End)]
    
print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2014-08-26
2017-02-26


In [4]:
# change to the weekly based on the celery weekly run

chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk = chunk.drop_duplicates()
    chunk=chunk[chunk['transaction_date']>=str(lapsed_date_begin)]
    chunk=chunk[chunk['transaction_date']<=str(prior_quarter_End)]
    
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()    

1 2019-11-09 13:23:43.509708
2 2019-11-09 13:25:31.077879
3 2019-11-09 13:27:10.796190
4 2019-11-09 13:29:13.376862
5 2019-11-09 13:31:14.990083
6 2019-11-09 13:33:35.058814
7 2019-11-09 13:35:37.031706
8 2019-11-09 13:37:55.543631
9 2019-11-09 13:40:27.209853
10 2019-11-09 13:42:55.456513
11 2019-11-09 13:45:43.224757
12 2019-11-09 13:48:21.183792
13 2019-11-09 13:51:14.527464
14 2019-11-09 13:54:36.404786
15 2019-11-09 13:58:21.194082


In [5]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
print(all_rewards_most_recent.shape)

del chunk
del dftrans_before_20180922
gc.collect()

(22431677, 2)


0

In [6]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [7]:
print("all_rewards_most_recent['transaction_date'].max()",all_rewards_most_recent['transaction_date'].max())

print("all_rewards_most_recent['transaction_date'].min()",all_rewards_most_recent['transaction_date'].min())


all_rewards_most_recent['transaction_date'].max() 2018-08-04
all_rewards_most_recent['transaction_date'].min() 2014-08-26


In [8]:
# Historical    
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recursive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2018,11,3)] # hard-code due to the 1st week of daily in item available
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>=datetime.date(2018,8,5)] # hard-code
historical_daily_df=historical_daily_df.sort_values("week_end_dt")
print(historical_daily_df.shape)
historical_daily_df.tail(3)

print("historical_daily_df['week_end_dt'].max(): ", historical_daily_df['week_end_dt'].max())
print("historical_daily_df['week_end_dt'].min(): ", historical_daily_df['week_end_dt'].min())

(13, 2)
historical_daily_df['week_end_dt'].max():  2018-11-03
historical_daily_df['week_end_dt'].min():  2018-08-11


In [9]:
prior_quarter_End

datetime.date(2018, 8, 4)

In [10]:
recent_complete_quarter_End

datetime.date(2018, 11, 3)

In [11]:
print(historical_daily_df.shape)
historical_daily_df.sort_values("week_end_dt")

(13, 2)


,file_path,week_end_dt
0,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-08-11
1,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-08-18
2,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-08-25
3,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-01
4,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-08
5,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-15
6,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-22
7,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-29
8,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-06
9,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-13


In [12]:
rew_df_after_recent_quarter_only=pd.DataFrame()

for file in historical_daily_df['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_recent_quarter_only=rew_df_after_recent_quarter_only.append(df)
    print(file,datetime.datetime.now())


/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180811.txt 2019-11-09 14:34:45.904731
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180818.txt 2019-11-09 14:35:05.419324
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180825.txt 2019-11-09 14:35:26.450817
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180901.txt 2019-11-09 14:35:47.097390
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180908.txt 2019-11-09 14:36:08.505018
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180915.txt 2019-11-09 14:36:28.928061
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180922.txt 2019-11-09 14:36:47.836099
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-11-09 14:37:14.739360


In [13]:
print(datetime.datetime.now())

2019-11-09 14:39:30.331295


In [14]:
# all_rewards_most_recent=all_rewards_most_recent.append(rew_df_after_20180929_before_quarter)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(rew_df_after_recent_quarter_only)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


0

In [15]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].astype(str)

# Change here to use unique dates and merge
all_rewards_most_recent_date=all_rewards_most_recent[['transaction_date_before_the_Quarter']].drop_duplicates()
all_rewards_most_recent_date['date']=pd.to_datetime(all_rewards_most_recent_date['transaction_date_before_the_Quarter'],format="%Y-%m-%d").dt.date
all_rewards_most_recent_date['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent_date['date']
all_rewards_most_recent_date['Month_to_pre_Quarter_End']=all_rewards_most_recent_date['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent_date['Group_before_the_Quarter']=np.where((all_rewards_most_recent_date['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent_date['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                       np.where(all_rewards_most_recent_date['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                                np.where((all_rewards_most_recent_date['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent_date['Month_to_pre_Quarter_End']<=12),"Active",
                                                                        np.where(all_rewards_most_recent_date['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                        )
                                                               )
                                                       )
all_rewards_most_recent_date=all_rewards_most_recent_date[['transaction_date_before_the_Quarter','Group_before_the_Quarter']]
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_date,on="transaction_date_before_the_Quarter",how="left")

In [16]:
recent_complete_quarter_End

datetime.date(2018, 11, 3)

In [17]:
all_rewards_most_recent['transaction_date_after_the_Quarter']=all_rewards_most_recent['transaction_date_after_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_after_the_Quarter']=all_rewards_most_recent['transaction_date_after_the_Quarter'].astype(str)

all_rewards_most_recent_date=all_rewards_most_recent[['transaction_date_after_the_Quarter']].drop_duplicates()
all_rewards_most_recent_date['date']=pd.to_datetime(all_rewards_most_recent_date['transaction_date_after_the_Quarter'],format="%Y-%m-%d").dt.date
all_rewards_most_recent_date['Days_to_recentt_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent_date['date']
all_rewards_most_recent_date['Month_to_recent_Quarter_End']=all_rewards_most_recent_date['Days_to_recentt_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent_date['Group_after_the_Quarter']=np.where((all_rewards_most_recent_date['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent_date['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                       np.where(all_rewards_most_recent_date['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                                np.where((all_rewards_most_recent_date['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent_date['Month_to_recent_Quarter_End']<=12),"Active",
                                                                        np.where(all_rewards_most_recent_date['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                        )
                                                               )
                                                       )

all_rewards_most_recent_date=all_rewards_most_recent_date[['transaction_date_after_the_Quarter','Group_after_the_Quarter']]
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_date,on="transaction_date_after_the_Quarter",how="left")

In [18]:
all_rewards_most_recent['Group_before_the_Quarter'].unique().tolist()

['Lapsed_13_48', 'Active', 'NotAvailable_Before_Quarter']

In [19]:
all_rewards_most_recent['Group_after_the_Quarter'].unique().tolist()

['Active', 'Lapsed_13_48', 'WD_48+']

In [20]:
all_rewards_most_recent[all_rewards_most_recent['Group_before_the_Quarter']=="Active"].shape

(15821968, 6)

In [21]:
print(datetime.datetime.now())
# stop here

2019-11-09 15:04:50.860830


In [22]:
RecentQuarter_sales_by_id=pd.DataFrame()
print(historical_daily_df.shape)
for file in historical_daily_df['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_sales=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales_recent_Quarter"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed"])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_recent_Quarter"})
    
    df=pd.merge(df_sales,df_trans,on="customer_id_hashed",how="outer")
    
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"

(13, 2)
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180811.txt 2019-11-09 15:05:38.341976
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180818.txt 2019-11-09 15:06:28.628952
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180825.txt 2019-11-09 15:07:18.825180
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180901.txt 2019-11-09 15:08:09.945672
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180908.txt 2019-11-09 15:09:01.126377
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180915.txt 2019-11-09 15:09:49.096383
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180922.txt 2019-11-09 15:10:36.138076
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-11-09 15:11:41

In [23]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent['Group_before_the_Quarter']=all_rewards_most_recent['Group_before_the_Quarter'].fillna("NotAvailable_Before_Quarter")
all_rewards_most_recent['Group_after_the_Quarter']=all_rewards_most_recent['Group_after_the_Quarter'].fillna("Active")

all_rewards_most_recent.shape

(23862131, 9)

In [ ]:
# change the 2018Q3 master

In [24]:
Recent_Quarter_new_sign_ups_files=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))

Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "Master" in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if ".txt" in x]

Recent_Quarter_new_sign_ups_files_df=pd.DataFrame({"file_path":Recent_Quarter_new_sign_ups_files})
Recent_Quarter_new_sign_ups_files_df['week_end_dt']=Recent_Quarter_new_sign_ups_files_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']<=str(recent_complete_quarter_End)]
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]
print(Recent_Quarter_new_sign_ups_files_df.shape)
Recent_Quarter_new_sign_ups_files_df.head(2)

(7, 2)


,file_path,week_end_dt
3,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-08-11
4,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-08-25


In [26]:
str(prior_quarter_End)

'2018-08-04'

In [27]:
all_new_sign_ups=pd.DataFrame()
for file in Recent_Quarter_new_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    all_new_sign_ups=all_new_sign_ups.append(df)
    
all_new_sign_ups=all_new_sign_ups[all_new_sign_ups['sign_up_date']>str(prior_quarter_End)]
all_new_sign_ups=all_new_sign_ups[all_new_sign_ups['sign_up_date']<=str(recent_complete_quarter_End)]

all_new_sign_ups=all_new_sign_ups.sort_values(['customer_id_hashed','sign_up_date'],ascending=[True,False])
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")
print(all_new_sign_ups.shape)



(995391, 2)


In [31]:
df_master_2018year=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                 compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
df_master_2018year=df_master_2018year[df_master_2018year['sign_up_date']>str(prior_quarter_End)]
df_master_2018year=df_master_2018year[df_master_2018year['sign_up_date']<=str(recent_complete_quarter_End)]
df_master_2018year=df_master_2018year.sort_values(['customer_id_hashed','sign_up_date'],ascending=[True,False])
df_master_2018year=df_master_2018year.drop_duplicates("customer_id_hashed")
print(df_master_2018year.shape)

(1472350, 2)


In [35]:
all_new_sign_ups=all_new_sign_ups.append(df_master_2018year)
all_new_sign_ups=all_new_sign_ups[all_new_sign_ups['sign_up_date']>str(prior_quarter_End)]
all_new_sign_ups=all_new_sign_ups[all_new_sign_ups['sign_up_date']<=str(recent_complete_quarter_End)]

all_new_sign_ups=all_new_sign_ups.sort_values(['customer_id_hashed','sign_up_date'],ascending=[True,False])
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")
print(all_new_sign_ups.shape)

all_new_sign_ups['NewRewards']="Recent_Quarter_New_Sign_Ups"


(1472358, 2)


In [36]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_new_sign_ups,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")

all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)

all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

all_rewards_most_recent.shape

(24326203, 11)

In [37]:
all_rewards_most_recent['Group_before_the_Quarter']=all_rewards_most_recent['Group_before_the_Quarter'].fillna("nan")
all_rewards_most_recent['Group_after_the_Quarter']=all_rewards_most_recent['Group_after_the_Quarter'].fillna("nan")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("nan")
all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("nan")

In [38]:
all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)


In [39]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()

output=pd.merge(output_id_count,output_id_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'],how="outer")
output

,Group_before_the_Quarter,Group_after_the_Quarter,NewRewards,Recent_Quarter_Shopping_Group,id_count,sales_recent_Quarter,trans_recent_Quarter
0,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,8623789,0.000000e+00,0.0
1,Active,Active,Old_Rewards_Members,Recent_Quarter_Shopped,5910570,4.853476e+08,14522273.0
2,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,19,0.000000e+00,0.0
3,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_Shopped,84,1.466149e+04,269.0
4,Active,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,1287506,0.000000e+00,0.0
5,Lapsed_13_48,Active,Old_Rewards_Members,Recent_Quarter_Shopped,514365,3.542193e+07,767392.0
6,Lapsed_13_48,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,6026419,0.000000e+00,0.0
7,Lapsed_13_48,Lapsed_13_48,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,1,0.000000e+00,0.0
8,Lapsed_13_48,WD_48+,Old_Rewards_Members,Recent_Quarter_No_Shop,68924,0.000000e+00,0.0
9,NotAvailable_Before_Quarter,Active,Old_Rewards_Members,Recent_Quarter_Shopped,422272,3.367350e+07,631490.0


In [40]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Migration_JL_status_change'

In [41]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))

output['inclusion/exclusion']=np.where((output['Group_before_the_Quarter']!="NotAvailable_Before_Quarter") & (output['NewRewards']=="Recent_Quarter_New_Sign_Ups"),"Exclusion","Inclusion")


In [42]:
output.to_csv("./BL_2018Q3_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [43]:
all_rewards_most_recent.to_csv("./BL_2018Q3_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [44]:
output['id_count'].sum()

24326203

In [45]:
all_rewards_most_recent.shape

(24326203, 11)

In [46]:
all_rewards_most_recent['customer_id_hashed'].nunique()

24326203

In [47]:
all_rewards_most_recent.shape

(24326203, 11)

In [48]:
all_rewards_most_recent.head(100)

,customer_id_hashed,transaction_date,transaction_date_before_the_Quarter,transaction_date_after_the_Quarter,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,trans_recent_Quarter,Recent_Quarter_Shopping_Group,sign_up_date,NewRewards
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,2017-01-19,2017-01-19,2018-10-30,Lapsed_13_48,Active,211.78,3.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
1,000002c86b9805b916f2a04b7a0031bb9ee03224e98221...,2014-12-01,2014-12-01,2014-12-01,Lapsed_13_48,Lapsed_13_48,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
2,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,2017-11-10,2017-11-10,2017-11-10,Active,Active,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
3,0000034fec3c2cbbf7a2b38fbf80d84abc72d8bed269c4...,2018-07-22,2018-07-22,2018-07-22,Active,Active,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
4,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,2018-07-14,2018-07-14,2018-10-15,Active,Active,11.10,1.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
...,...,...,...,...,...,...,...,...,...,...,...
95,00004d161929f36dc892fae11bff1c106d699e51daf5e0...,2018-07-08,2018-07-08,2018-10-27,Active,Active,14.65,1.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
96,00004d52e7c3521618e946bd42f159155972209a11b63d...,2016-09-04,2016-09-04,2016-09-04,Lapsed_13_48,Lapsed_13_48,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
97,00004df5c8fbf830dc526ac784ca62493d9adcc6bc8204...,2018-01-22,2018-01-22,2018-01-22,Active,Active,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
98,00004eb78a416ee5bf936f1fdb9d8504ea4694dae3aa95...,2018-06-19,2018-06-19,2018-06-19,Active,Active,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members


In [49]:
all_rewards_most_recent[all_rewards_most_recent['Group_before_the_Quarter']=="Active"].shape

(15821968, 11)